# API Nakala

In [ ]:
# ANR experts
# author: @sardinecan
# date: 2022-12
# description: this Julia notebook allows to interact with the Nakala's API
# licence: CC-0

# @todo : gestion des erreurs (réponses server) ?
# @todo : écrire un fichier de log pour récupérer les identifiants des ressources crées.

## Packages

In [ ]:
using Pkg
path = @__DIR__
Pkg.activate(path)
Pkg.status()

using CSV
using DataFrames
using HTTP
using JSON
using Dates
using FileIO
using Images

## Identifiants

In [ ]:
credentials = CSV.read(joinpath(path, "credentials.csv"), DataFrame, header=1) #liste des utilisateurs
user = "tnakala" #choix de l'utilisateur (api test = tnakala)
usrCredentials = filter(:user => n -> n == user, credentials) #récupération des identifiants
apiKey = usrCredentials[1, :apikey] #clé API

## API
Pour travailler avec l'API test, modifiez la valeur de la variable `apitest` par `true`

In [ ]:
apitest = true

if apitest == true
  apiurl = "https://apitest.nakala.fr"
else
  apiurl = "https://api.nakala.fr"
end

## Librairie de fonctions

In [ ]:
newARGS = apiKey #passe l'apikey dans la librairie
include("functions.jl")

## Collections

### Création d'une collection

Une fois les identifiants chargés et l'API sélectionnée, entrez simplement un nom de collection pour la créer.

In [ ]:
collectionName = "testCollection"
postCollection(collectionName)

### Ajouter des données à une collection

In [ ]:
collectionIdentifier = "10.34847/nkl.5c519k06"
datas = [
  "nakalaDataId_1",
  "nakalaDataId_2",
  "nakalaDataId_3"
]

postDatasToCollection(collectionIdentifier, datas)

### Supprimer des données d'une collection

In [ ]:
collectionIdentifier = ""
datas = [
  "nakalaDataId_1",
  "nakalaDataId_2",
  "nakalaDataId_3"
]

deleteDatasFromCollection(collectionIdentifier, datas)

## Utilisateurs

### Mes Informations

In [ ]:
getUserInfo() #retourne un dictionnaire

# pour accéder aux entrées du dictionnaire
#username = getUserInfo()["username"]
#userGroupId = getUserInfo()["userGroupId"]

### Ressources utilisateur

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
postUserDatas(scope[1])

### Lister les données privées/publiques

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ] #[ "pending", "published" ]

postUserDatas(scope[1], status[2])

### Changer le statut d'une donnée

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"], ["moderated"] ]

datasToModify = get(postUserDatas(scope[1], status[1]), "data", "")
datasToModifyIds = [get(data, "identifier", "") for data in datasToModify]
newStatus = status[2][1] #pour récupérer la valeur textuelle du vecteur.

[putDataStatus(data, newStatus) for data in datasToModifyIds]

### Lister les données (titre - id)

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ]

datas = get(postUserDatas(scope[1]), "data", "nothing")

getDatasResume(datas)

### Chercher une donnée précise

In [ ]:
title = "test"

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ]

datas = get(postUserDatas(scope[1], status[2], title), "data", "nothing")
datas[1] # il peut y avoir plusieurs résultats

### Télécharger les images d'une donnée

In [ ]:
title = "Test temporal"

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ]

datas = get(postUserDatas(scope[1], status[2], title), "data", "nothing")
data = datas[1]

# peut fonctionner avec d'autres formats de données (mais pas toutes, voir https://juliaio.github.io/FileIO.jl/stable/registry/#Registry-table)

downloadFiles(data, title)

### Chercher un fichier précis dans une donnée

In [ ]:
title = "test"
filenames = ["The_Earth_seen_from_Apollo_17.jpg"]

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ]

datas = get(postUserDatas(scope[1], status[2], title), "data", "nothing")
data = datas[1]

getFilesUrlFromData(data, filenames)

## Données

### Publier une donnée avec des fichiers

In [ ]:
path = joinpath(@__DIR__, "datasToSubmit") # chemin vers le dossier contenant les données à envoyer
directories = listFile(path)
[submitDataFromFolder(path, directory) for directory in directories]

### Ajouter des fichiers à une données

In [ ]:
dataIdentifier = "10.34847/nkl.5b62qk50" # identifier de la données à compléter
path = joinpath(@__DIR__, "filesToUpload") # chemin vers le dossier contenant les fichiers à ajouter
postedfiles = [postFile(joinpath(path, file)) for file in readdir(path)]

[addFileToData(dataIdentifier, file) for file in postedfiles]